# Homework

In [1]:
import graphlab

In [45]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')
graphlab.canvas.set_target('ipynb')

In [46]:
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


** 1. Computing summary statistics of the data:** Sketch summaries are techniques for computing summary statistics of data very quickly. In GraphLab Create, SFrames and SArrays include a method: `.sketch_summary()` which computes such summary statistics. Using the training data, compute the sketch summary of the `label` column and interpret the results. What’s the least common category in the training data?

In [3]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


** 2. Creating category-specific image retrieval models:** In most retrieval tasks, the data we have is unlabeled, thus we call these unsupervised learning problems. However, we have labels in this image dataset, and will use these to create one model for each of the 4 image categories, `{‘dog’,’cat’,’automobile’,bird’}`. To start, follow these steps:

* Split the SFrame with the training data into 4 different SFrames. Each of these will contain data for 1 of the 4 categories above. Hint: if you use a logical filter to select the rows where the `label` column equals `dog`, you can create an SFrame with only the data for images labeled `dog`.

* Similarly to the image retrieval notebook you downloaded, you are going to create a nearest neighbor model using the `deep_features` as the features, but this time create one such model for each category, using the corresponding subset of the training_data. You can call the model with the `dog` data the `dog_model`, the one with the `cat` data the `cat_model`, as so on.

Using these models, answer the following questions

In [4]:
dog_data = image_train[image_train['label'] == 'dog']
cat_data = image_train[image_train['label'] == 'cat']
bird_data = image_train[image_train['label'] == 'bird']
automobile_data = image_train[image_train['label'] == 'automobile']

lengths = (len(automobile_data), len(cat_data), len(dog_data), len(bird_data))

print '+-------+------------+-----+-----+------+'
print '| value | automobile | cat | dog | bird |'
print '+-------+------------+-----+-----+------+'
print '| count |    %d     | %d | %d | %d  |' % lengths
print '+-------+------------+-----+-----+------+'

+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


In [5]:
dog_model = graphlab.nearest_neighbors.create(dog_data, features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

In [6]:
cat_model = graphlab.nearest_neighbors.create(cat_data, features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

In [7]:
bird_model = graphlab.nearest_neighbors.create(bird_data, features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

In [8]:
automobile_model = graphlab.nearest_neighbors.create(automobile_data, features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

* What is the nearest `cat` labeled image in the training data to the first image of the first image in the test data?

In [9]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'], 'id')

cat = image_test[0:1]
cat.show()

In [41]:
cat_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 9.606ms      |

| Done         |         | 100         | 51.107ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16289,34.623719208,1
0,45646,36.0068799284,2
0,32139,36.5200813436,3
0,25713,36.7548502521,4
0,331,36.8731228168,5


In [52]:
image_train[image_train['id'] == 16289]['image'].show()

In [11]:
cat_query = cat_model.query(cat)
get_images_from_ids(cat_query)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 9.684ms      |

| Done         |         | 100         | 95.099ms     |

+--------------+---------+-------------+--------------+

id,image,label,deep_features,image_array
331,Height: 32 Width: 32,cat,"[0.0, 0.0,0.510963916779, 0.0, ...","[45.0, 65.0, 92.0, 72.0,95.0, 110.0, 106.0, ..."
16289,Height: 32 Width: 32,cat,"[0.964287519455, 0.0,0.0, 0.0, 1.12515509129, ...","[215.0, 219.0, 231.0,215.0, 219.0, 232.0, ..."
25713,Height: 32 Width: 32,cat,"[0.536971271038, 0.0,0.0, 0.0894458889961, ...","[228.0, 222.0, 236.0,224.0, 213.0, 222.0, ..."
32139,Height: 32 Width: 32,cat,"[1.29409468174, 0.0, 0.0,0.513800263405, ...","[217.0, 220.0, 205.0,221.0, 227.0, 218.0, ..."
45646,Height: 32 Width: 32,cat,"[0.983677506447, 0.0,0.0, 0.0, 0.0, ...","[51.0, 42.0, 26.0, 56.0,47.0, 31.0, 59.0, 50.0, ..."


In [12]:
cat_neighbors = get_images_from_ids(cat_query)
cat_neighbors['image'].show()

* What is the nearest `dog` labeled image in the training data to the cat image above (the first image in the test data)?

In [53]:
dog_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 10.741ms     |

| Done         |         | 100         | 53.129ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16976,37.4642628784,1
0,13387,37.5666832169,2
0,35867,37.6047267079,3
0,44603,37.7065585153,4
0,6094,38.5113254907,5


In [54]:
image_train[image_train['id'] == 16976]['image'].show()

In [14]:
cat_query2 = dog_model.query(cat)
get_images_from_ids(cat_query2)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 8.159ms      |

| Done         |         | 100         | 51.46ms      |

+--------------+---------+-------------+--------------+

id,image,label,deep_features,image_array
6094,Height: 32 Width: 32,dog,"[0.470533549786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[91.0, 98.0, 71.0, 138.0,123.0, 63.0, 135.0, ..."
13387,Height: 32 Width: 32,dog,"[0.366494178772, 0.0,0.0, 0.0, 0.0, 0.0, ...","[255.0, 255.0, 255.0,255.0, 255.0, 255.0, ..."
16976,Height: 32 Width: 32,dog,"[0.755595386028, 0.0,0.0, 0.0, 0.0, 0.0, ...","[16.0, 17.0, 11.0, 18.0,19.0, 13.0, 20.0, 21.0, ..."
35867,Height: 32 Width: 32,dog,"[0.305321395397, 0.0,0.0, 0.0, 0.0, 0.0, ...","[101.0, 93.0, 9.0, 93.0,88.0, 9.0, 90.0, 86.0, ..."
44603,Height: 32 Width: 32,dog,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 11.2646656036, 0.0, ...","[8.0, 25.0, 9.0, 29.0,39.0, 22.0, 66.0, 75.0, ..."


In [15]:
cat_neighbors2 = get_images_from_ids(cat_query2)
cat_neighbors2['image'].show()

** 3. A simple example of nearest-neighbors classification:** When we queried a nearest neighbors model, the `distance` column in the table above shows the computed distance between the input and each of the retrieved neighbors. In this question, you will use these distances to perform a classification task, using the idea of a nearest-neighbors classifier.

In [16]:
knn_model = graphlab.nearest_neighbors.create(image_train, features=['deep_features'], label='id')

Starting brute force nearest neighbors model training.

* For the first image in the test data (`image_test[0:1]`), which we used above, compute the mean distance between this image at its 5 nearest neighbors that were labeled `cat` in the training data (similarly to what you did in the previous question).

In [17]:
cat = image_test[0:1]
knn_cat_query = knn_model.query(cat)
sum(knn_cat_query['distance']) / len(knn_cat_query)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 14.329ms     |

| Done         |         | 100         | 133.822ms    |

+--------------+---------+-------------+--------------+

36.15573070978294

In [18]:
knn_cat_query2 = cat_model.query(cat)
sum(knn_cat_query2['distance']) / len(knn_cat_query2)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 8.741ms      |

| Done         |         | 100         | 52.617ms     |

+--------------+---------+-------------+--------------+

36.15573070978294

* Similarly, for the first image in the test data (`image_test[0:1]`), which we used above, compute the mean distance between this image at its 5 nearest neighbors that were labeled `dog` in the training data (similarly to what you did in the previous question).

In [19]:
knn_cat_query3 = dog_model.query(cat)
sum(knn_cat_query3['distance']) / len(knn_cat_query3)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 7.345ms      |

| Done         |         | 100         | 46.752ms     |

+--------------+---------+-------------+--------------+

37.77071136184157

** 4. Computing nearest neighbors accuracy using SFrame operations:** A nearest neighbor classifier predicts the label of a point as the most common label of its nearest neighbors. In this question, we will measure the accuracy of a 1-nearest-neighbor classifier, i.e., predict the output as the label of the nearest neighbor in the training data. Although there are simpler ways of computing this result, we will go step-by-step here to introduce you to more concepts in nearest neighbors and SFrames, which will be useful later in this Specialization.

* **Training models:** For this question, you will need the nearest neighbors models you learned above on the training data, i.e., the `dog_model`, `cat_model`, `automobile_model` and `bird_model`.
* **Spliting test data by label:** Above, you split the train data SFrame into one SFrame for images labeled `dog`, another for those labeled `cat`, etc. Now, do the same for the test data. You can call the resulting SFrames

```python
image_test_cat, image_test_dog, image_test_bird, image_test_automobile
```

* **Finding nearest neighbors in the training set for each part of the test set:** Thus far, we have queried, e.g.,

```python
dog_model.query()
```

our nearest neighbors models with a single image as the input, but you can actually query with a whole set of data, and it will find the nearest neighbors for each data point. Note that the input index will be stored in the `query_label` column of the output SFrame.

Using this knowledge find the closest neighbor in to the dog test data using each of the trained models, e.g.,

```python
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)
```

finds 1 neighbor (that’s what k=1 does) to the dog test images (`image_test_dog`) in the cat portion of the training data (used to train the `cat_model`).

Now, do this for every combination of the labels in the training and test data.

In [20]:
image_test_cat = image_test[image_test['label'] == 'cat']
image_test_dog = image_test[image_test['label'] == 'dog']
image_test_bird = image_test[image_test['label'] == 'bird']
image_test_automobile = image_test[image_test['label'] == 'automobile']

In [21]:
dog_cat_neighbors = cat_model.query(image_test_dog, k=1)
dog_dog_neighbors = dog_model.query(image_test_dog, k=1)
dog_bird_neighbors = bird_model.query(image_test_dog, k=1)
dog_automobile_neighbors = automobile_model.query(image_test_dog, k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 63000   | 12.3772     | 327.162ms    |

| Done         | 509000  | 100         | 351.901ms    |

+--------------+---------+-------------+--------------+

* **Create an SFrame with the distances from `dog` test examples to the respective nearest neighbors in each class in the training data:** The `distance` column in `dog_cat_neighbors` above contains the distance between each `dog` image in the test set and its nearest `cat` image in the training set. The question we want to answer is how many of the test set `dog` images are closer to a `dog` in the training set than to a `cat`, `automobile` or `bird`. So, next we will create an SFrame containing just these distances per data point. The goal is to create an SFrame called `dog_distances` with 4 columns:


1. `dog_distances[‘dog-dog’]` ---- storing `dog_dog_neighbors[‘distance’]`

2. `dog_distances[‘dog-cat’]` ---- storing `dog_cat_neighbors[‘distance’]`

3. `dog_distances[‘dog-automobile’]` ---- storing `dog_automobile_neighbors[‘distance’]`

4. `dog_distances[‘dog-bird’]` ---- storing `dog_bird_neighbors[‘distance’]`

In [33]:
dog_distances = graphlab.SFrame({
        'dog-cat': dog_cat_neighbors['distance'],
        'dog-dog': dog_dog_neighbors['distance'],
        'dog-bird': dog_bird_neighbors['distance'],
        'dog-automobile': dog_automobile_neighbors['distance']})

dog_distances.head()

dog-automobile,dog-bird,dog-cat,dog-dog
41.9579761457,41.7538647304,36.4196077068,33.4773590373
46.0021331807,41.3382958925,38.8353268874,32.8458495684
42.9462290692,38.6157590853,36.9763410854,35.0397073189
41.6866060048,37.0892269954,34.5750072914,33.9010327697
39.2269664935,38.272288694,34.778824791,37.4849250909
40.5845117698,39.1462089236,35.1171578292,34.945165344
45.1067352961,40.523040106,40.6095830913,39.0957278345
41.3221140974,38.1947918393,39.9036867306,37.7696131032
41.8244654995,40.1567131661,38.0674700168,35.1089144603
45.4976929401,45.5597962603,42.7258732951,43.2422832585


* **Computing the number of correct predictions using 1-nearest neighbors for the dog class:** Now that you have created the SFrame `dog_distances`, you will learn to use the method `.apply()` on this SFrame to iterate line by line and compute the number of `dog` test examples where the distance to the nearest `dog` was lower than that to the other classes. You will do this in three steps:

I. Consider one row of the SFrame `dog_distances`. Let’s call this variable row. You can access each distance by calling, for example, `row['dog-cat']` which, in example table above, will have value equal to 36.4196077068 for the first row. Create a function starting with 

```python
def is_dog_correct(row):
```
which returns 1 if the value for `row[‘dog-dog’]` is lower than that of the other columns, and 0 otherwise. That is, returns 1 if this row is correctly classified by 1-nearest neighbors, and 0 otherwise.

In [34]:
def is_dog_correct(row):
    return 1 if row['dog-dog'] <= min(row.values()) else 0

II. Using the function `is_dog_correct(row)`, you can check if 1 row is correctly classified. Now, you want to count how many rows are correctly classified. You could do a for loop iterating through each row and applying the function `is_dog_correct(row)`. This method will be really slow, because the SFrame is not optimized for this type of operation.

Instead, we will use the `.apply()` method to iterate the function is_dog_correct for each row of the SFrame. Read about using the `.apply()` method [here](https://turi.com/learn/userguide/sframe/tabular-data.html).

III. Computing the number of correct predictions for `dog`: You can now call:

```python
dog_distances.apply(is_dog_correct)
```

which will return an SArray (a column of data) with a 1 for every correct row and a 0 for every incorrect one. You can call:

```python
.sum()
```

In [37]:
dog_distances.apply(is_dog_correct)

dtype: int
Rows: 1000
[1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, ... ]

In [38]:
dog_distances.apply(is_dog_correct).sum()

678

In [39]:
dog_distances.apply(is_dog_correct).sum() / float(len(dog_distances))

0.678